#Problema del negocio

El caso de estudio está enfocado en una aseguradora que evalúa pólizas vehicualres para lo cual se ha generado un dataset de **64 695 clientes** caracterizados con un total de **20 variables** entre categóricas y numéricas.

En el presente caso nos enfrentamos a un problema de **clasificación** donde se busca predecir una variable denominada **target** de caracter **binario**:

1.   Siniestroso: Un cliente cuyas características predicen con una alta probabilidad que podría tener un accidente vehicular
2.   No Siniestroso: Un cliente cuyas características predicen que es muy poco probable que sufra un accidente vehicular

Por el **tipo de target binario** y el problema de **clasificación** se ha decidido abordar el presente problema con un **modelo de regresión logística** ya que utiliza una función **sigmoide** que devuelve solo **dos valores de predicción** en función a un punto de corte.


#I. Entendiendo los datos

He subido el dataset a mi [github](https://github.com/javalpe/datasets) para poder leer el archivo en formato **raw**

In [0]:
url="https://raw.githubusercontent.com/javalpe/datasets/master/siniestros_data.csv"

He cargado la librería **pandas** para leer el archivo csv

In [2]:
import pandas as pd

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


Ahora guardo el dataframe del archivo en una variable denominada **datos**

In [0]:
datos=pd.read_csv(url)

A modo de comprobación ejecuto el método **shape** para visualizar el **número de filas y columnas**

In [4]:
datos.shape

(64695, 21)

Ahora ejecuto el método **info** para visualizar el tipo de dato en cada columna

In [5]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64695 entries, 0 to 64694
Data columns (total 21 columns):
Unnamed: 0         64695 non-null int64
Ubicacion          64695 non-null object
Uso                64695 non-null object
Clase              64695 non-null object
Asientos           64695 non-null int64
Canal_E            64695 non-null object
PERDIDA_3          64695 non-null float64
OK_3               64695 non-null float64
NUM_TC_3           64695 non-null int64
MAX_LINEA_SOL_3    64695 non-null float64
MAX_LINEA_DOL_3    64695 non-null float64
Edad               64695 non-null int64
EdadAuto           64695 non-null int64
Gama               64695 non-null object
sexo_Mod           64695 non-null object
sueldo_bruto       64695 non-null int64
target             64695 non-null object
Renov_SOAT         64695 non-null int64
media_ok           64695 non-null float64
deuda_dol          64695 non-null float64
deuda_sol          64695 non-null float64
dtypes: float64(7), int64(7), 

Se pueden extraer los siguientes hallazgos: 

1.   Varias columnas cuentan con datos de tipo '**categóricos**. Estos datos serán transformados más adelante a tipo numérico
2.   **No existen valores nulos o perdidos** 

#II. Preprocesamiento de los datos

In [6]:
datos.target.value_counts()

No_Siniestroso    64226
Siniestroso         469
Name: target, dtype: int64

In [0]:
datos["target"] = datos["target"].map({"No_Siniestroso":0,"Siniestroso":1})

En el caso de la variable **Ubicacion**, acorde a una [nota periodística](https://rpp.pe/lima/actualidad/lima-ocho-distritos-registran-mayor-cantidad-de-accidentes-de-transito-noticia-976292?ref=rpp), debemos ponderar la ubicación según la cantidad potencial de accidentes en la zona de Lima

In [0]:
datos["Ubicacion"] = datos["Ubicacion"].map({"LimaModerna1":1,"LimaEste":2,"LimaModerna2":1,"LimaCentro":3,"Provincia":1,"LimaSur":4,"LimaNorte":4,"Callao":2,"LimaND":1,"LimaProvincia":1})

En el caso de la variable **Uso** distinguimos el particular o personal del transporte. Finalmente cualquier uso distintos a estos se codificará como cero (0)

In [0]:
datos["Uso"]=datos["Uso"].map({"PARTICULAR":1,"CARGA / TRANSPORTE":2,"TAXI":2,"ESCOLAR":0,"PERSONAL":1,"PUBLICO":0,"INTERPROVINCIAL":0})

Para la variable **Clase** tomaremos como referencia **AFOCAT** que [reporta](https://www.afocatlimam.org/Principal.php) el indice de siniestralidad de acuerdo al tipo de vehículo

In [0]:
datos["Clase"]=datos["Clase"].map({"AUTOMOVIL":0.3144,"CAMIONETA RURAL":0.3265,"CAMIONETA PICK UP":0,"STATION WAGON":0.0854,"CAMIONETA PANEL":0,"CAMION":0,"OTROS":0})

Para la variable **sexo_Mod** tomaremos como referencia el [informe](https://www.inei.gob.pe/media/MenuRecursivo/boletines/02-informe-tecnico-n02_estadisticas-seguridad-ciudadana_set2018-feb2019.pdf) de seguridad ciudadana de **INEI** que ilustra con datos una **mayor incidencia de accidentes en hombres que mujeres**

In [0]:
datos["sexo_Mod"]=datos["sexo_Mod"].map({"M":3,"F":1})

Finalmente para las variables **Canal_E** y **Gama** simplemente realizaremos un **target encoding** para convertirlas a números

In [0]:
mean_Canal_E=datos.groupby('Canal_E')['target'].mean()
datos['Canal_E'] = datos['Canal_E'].map(mean_Canal_E)

In [0]:
mean_Gama=datos.groupby('Gama')['target'].mean()
datos['Gama'] = datos['Gama'].map(mean_Gama)

#III. Modelamiento

Definimos **dos nuevas variables**:
1.   **X**: Las variables predictoras de nuestro modelo
2.   **y**: La variable objetivo de nuestro modelo

In [0]:
X = datos.drop("target", axis = 1)
y = datos.target

**Dividimos la data entre test y entrenamiento** utilizando el método **X_train_test_split**

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, stratify=y)

Para el presente caso con **target desbalanceado** necesitamos **instalar la librería imblearn**

In [17]:
pip install imblearn

#Balanceando la data usando SMOTE

Para abordar el desbalanceo utilizando **SMOTE** importamos el método **SMOTE** de la librería **over_sampling**

In [0]:
from imblearn.over_sampling import SMOTE
smt = SMOTE()

**Transformamos los datos de entrenamiento** utilizando el método **fit_simple** con la librería **SMOTE** (smt)

In [0]:
X_train, y_train = smt.fit_sample(X_train, y_train)

**Comprobamos efectivamente que ahora sí está balenceado el target (y)** importando la librería **numpy** y utilizando el método **bincount**

In [24]:
import numpy as np
np.bincount(y_train)

array([48169, 48169])

Para ejecutar el modelo de **clasificación** importamos la librería **Logistic Regression**

In [0]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

**Entrenamos nuestro modelo** con los datos de entrenamiento **y nuestro target ahora sí balanceado**

In [26]:
lr.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

**Generamos nuestras predicciones** y guardamos nuestro resultado en la variable **y_pred**

In [0]:
y_pred = lr.predict(X_test)

#IV. Evaluando nuestro modelo utilizando SMOTE

Para **evaluar** nuestro modelo vamos a realizar **tres métricas**:
1.   **confusion_matrix**: donde comparamos las predicciones obtenidas por el modelo versus cada uno de los target reales (en este caso 0 y 1) 
2.   **accuracy_score**: donde contamos el total de coincidencias entre predicciones y target reales (en porcentaje)
3.   **recall_score**: donde solo comparamos las predicciones versus el target = 1 (true)

In [28]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[8631, 7426],
       [  45,   72]])

In [29]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.5380858167429208

In [30]:
from sklearn.metrics import recall_score
recall_score(y_test, y_pred)

0.6153846153846154

#Balanceando la data utilizando NearMiss

Para abordar el desbalanceo utilizando **NearMiss** importamos el método **NearMiss** de la librería **over_sampling**

In [0]:
from imblearn.under_sampling import NearMiss
nr = NearMiss()

**Transformamos los datos de entrenamiento** utilizando el método **fit_simple** con la librería **NearMiss** (nr)

In [0]:
X_train, y_train = nr.fit_sample(X_train, y_train)

**Comprobamos efectivamente que ahora sí está balenceado el target (y)** importando la librería **numpy** y utilizando el método **bincount**

In [33]:
import numpy as np
np.bincount(y_train)

array([48169, 48169])

Para ejecutar el modelo de **clasificación** importamos la librería **Logistic Regression**

In [0]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

**Entrenamos nuestro modelo** con los datos de entrenamiento **y nuestro target ahora sí balanceado**

In [35]:
lr.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

**Generamos nuestras predicciones** y guardamos nuestro resultado en la variable **y_pred**

In [0]:
y_pred = lr.predict(X_test)

#IV. Evaluando nuestro modelo utilizando NearMiss

Para **evaluar** nuestro modelo vamos a realizar **tres métricas**:
1.   **confusion_matrix**: donde comparamos las predicciones obtenidas por el modelo versus cada uno de los target reales (en este caso 0 y 1) 
2.   **accuracy_score**: donde contamos el total de coincidencias entre predicciones y target reales (en porcentaje)
3.   **recall_score**: donde solo comparamos las predicciones versus el target = 1 (true)

In [37]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[8710, 7347],
       [  46,   71]])

In [38]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.5429083714603685

In [39]:
from sklearn.metrics import recall_score
recall_score(y_test, y_pred)

0.6068376068376068